In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!git clone https://github.com/martiansideofthemoon/squash-generation.git
!cp -r squash-generation/pytorch-pretrained-BERT/pytorch_pretrained_bert .
!pip install -r squash-generation/requirements.txt

Cloning into 'squash-generation'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 375 (delta 1), reused 1 (delta 0), pack-reused 369
Receiving objects: 100% (375/375), 397.70 KiB | 4.85 MiB/s, done.
Resolving deltas: 100% (177/177), done.
     |████████████████████████████████| 102kB 2.8MB/s 
     |████████████████████████████████| 5.7MB 8.7MB/s 
     |████████████████████████████████| 1.7MB 42.0MB/s 
     |████████████████████████████████| 133kB 37.1MB/s 
     |████████████████████████████████| 5.6MB 4.7MB/s 
     |████████████████████████████████| 163kB 58.5MB/s 
     |████████████████████████████████| 552kB 51.9MB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 2.2MB 46.0MB/s 
     |████████████████████████████████| 2.8MB 43.1MB/s 
     |████████████████████████████████| 133kB 49.1MB/s

In [0]:
import os
import math
import logging
from pprint import pformat
from collections import defaultdict
from itertools import chain

import torch
import torch.nn.functional as F

from torch.nn.parallel import DistributedDataParallel
from torch.utils.data import DataLoader, TensorDataset
from ignite.engine import Engine, Events
from ignite.handlers import ModelCheckpoint
from ignite.metrics import Accuracy, Loss, MetricsLambda, RunningAverage
from ignite.contrib.handlers import ProgressBar, PiecewiseLinear
from ignite.contrib.handlers.tensorboard_logger import TensorboardLogger, OutputHandler, OptimizerParamsHandler
from pytorch_pretrained_bert import (OpenAIAdam, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer,
                                     GPT2LMHeadModel, GPT2Tokenizer, WEIGHTS_NAME, CONFIG_NAME)

import pytorch_pretrained_bert.modeling_gpt2 as gpt2

#from arguments import parser
#from dataloader import get_dataset

def top_filtering(logits, top_k=0, top_p=0.0, threshold=-float('Inf'), filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k, top-p (nucleus) and/or threshold filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k: <=0: no filtering, >0: keep only top k tokens with highest probability.
            top_p: <=0.0: no filtering, >0.0: keep only a subset S of candidates, where S is the smallest subset
                whose total probability mass is greater than or equal to the threshold top_p.
                In practice, we select the highest probability tokens whose cumulative probability mass exceeds
                the threshold top_p.
            threshold: a minimal threshold to keep logits
    """
    assert logits.dim() == 1  # Only work for batch size 1 for now - could update but it would obfuscate a bit the code
    top_k = min(top_k, logits.size(-1))
    if top_k > 0:
        # Remove all tokens with a probability less than the last token in the top-k tokens
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        # Compute cumulative probabilities of sorted tokens
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probabilities = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probabilities > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # Back to unsorted indices and set them to -infinity
        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value

    indices_to_remove = logits < threshold
    logits[indices_to_remove] = filter_value

    return logits

In [0]:
from collections import namedtuple
import random
args ={
    "max_length":45,
    "min_length":1,
    "top_k":10,
    "top_p":0.9,
    "device":"cpu",
    "temperature":0.7,
    "no_sample":False,
    "seed":42,
    "checkpoint_name":"pytorch_model.bin",
    "model_folder" :"models/question-generation/harvest"
}
args = namedtuple("GenricDict",args.keys())(**args)
random.seed(args.seed)
torch.random.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
SPECIAL_TOKENS = [
    "<bos>", "<eos>", "<context>","<question>","<answer>","<pad>"
]
MODEL_INPUTS = ["input_ids", "lm_labels", "token_type_ids"]
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.set_special_tokens(SPECIAL_TOKENS)
bos, eos, context,question_id,answer_id,pad = \
          tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)

special_tokens_ids = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)

gpt2.WEIGHTS_NAME = "checkpoint_mymodel_120000.bin"
model = GPT2LMHeadModel.from_pretrained("/content/drive/My Drive/harvest/")
model.set_num_special_tokens(len(SPECIAL_TOKENS))
model.eval()